In [1]:
from pyspark.sql.functions import *

In [2]:
df = spark.read.load('data/complete.csv',
                     format="csv", inferSchema="true", header="true")

In [3]:
df.schema

StructType(List(StructField(category,StringType,true),StructField(date,TimestampType,true),StructField(Wt Range,StringType,true),StructField(Avg Wt,IntegerType,true),StructField(Price Range,StringType,true),StructField(Avg Price,DoubleType,true),StructField(Extra,StringType,true)))

In [4]:
df.take(5)

[Row(category='Slaughter Cows Boner 80-85%', date=datetime.datetime(2019, 4, 25, 0, 0), Wt Range='1000-1100', Avg Wt=1050, Price Range='50.00-54.00', Avg Price=51.9, Extra=None),
 Row(category='Slaughter Cows Boner 80-85%', date=datetime.datetime(2019, 4, 25, 0, 0), Wt Range='1235-1390', Avg Wt=1318, Price Range='50.00-51.00', Avg Price=50.34, Extra=None),
 Row(category='Slaughter Cows Boner 80-85%', date=datetime.datetime(2019, 4, 25, 0, 0), Wt Range='1335-1350', Avg Wt=1342, Price Range='48.00-49.00', Avg Price=48.5, Extra='Low Dressing'),
 Row(category='Slaughter Cows Lean 85-90%', date=datetime.datetime(2019, 4, 25, 0, 0), Wt Range='960-1100', Avg Wt=1064, Price Range='45.00-50.00', Avg Price=47.88, Extra=None),
 Row(category='Slaughter Cows Lean 85-90%', date=datetime.datetime(2019, 4, 25, 0, 0), Wt Range='880-1100', Avg Wt=944, Price Range='38.00-44.00', Avg Price=40.46, Extra='Low Dressing')]

In [5]:
df.select("category").distinct().show(20, False)

+---------------------------------------+
|category                               |
+---------------------------------------+
|Cow-Calf Pairs Medium and Large 1-2    |
|Slaughter Bulls Y.G. 1                 |
|Slaughter Cows Boner 80-85%            |
|Feeder Bulls Medium and Large 1        |
|Bred Cows Medium and Large 2-3 per head|
|Feeder Heifers Medium and Large 2      |
|Bred Cows Medium and Large 1-2 per head|
|Slaughter Cows Lean 85-90%             |
|Feeder Bulls Medium and Large 3        |
|Feeder Heifers Medium and Large 1      |
|Feeder Heifers Medium and Large 3      |
|Feeder Bulls Medium and Large 2        |
+---------------------------------------+



In [6]:
# I want to convert the wt range to a high wt and low wt, and the price range to a high price and low price
withPriceCols = df.withColumn("priceRange", split(col("Price Range"), '-').cast("array<int>")) \
                    .withColumn("lowPrice", array_min(col("priceRange"))) \
                    .withColumn("highPrice", array_max(col("priceRange"))) \
                    .drop("Price Range") \
                    .drop("priceRange")

auctionResults = withPriceCols.withColumn("weightRange", split(col("Wt Range"), '-').cast("array<int>")) \
                                .withColumn("lowWeight", array_min(col("weightRange"))) \
                                .withColumn("highWeight", array_max(col("weightRange"))) \
                                .drop("Wt Range") \
                                .drop("weightRange")

In [7]:
auctionResults.show(1)

+--------------------+-------------------+------+---------+-----+--------+---------+---------+----------+
|            category|               date|Avg Wt|Avg Price|Extra|lowPrice|highPrice|lowWeight|highWeight|
+--------------------+-------------------+------+---------+-----+--------+---------+---------+----------+
|Slaughter Cows Bo...|2019-04-25 00:00:00|  1050|     51.9| null|      50|       54|     1000|      1100|
+--------------------+-------------------+------+---------+-----+--------+---------+---------+----------+
only showing top 1 row



In [8]:
auctionResults.schema

StructType(List(StructField(category,StringType,true),StructField(date,TimestampType,true),StructField(Avg Wt,IntegerType,true),StructField(Avg Price,DoubleType,true),StructField(Extra,StringType,true),StructField(lowPrice,IntegerType,true),StructField(highPrice,IntegerType,true),StructField(lowWeight,IntegerType,true),StructField(highWeight,IntegerType,true)))